<center>
<h1><b>Institute of Computer Technology</br>
B. Tech Computer Science and Engineering</b></h1>
<hr>
<ul type=None><li>Prajapati Yash P. (20162121023) [BDA-63]<br></li>
<li>Krupal M. Patel (20162121007) [BDA-62]</li></ul><hr>
<b>Subject:</b> Predictive Modeling
<hr>PROJECT</h3> - Predicting Depression, Anxiety & Stress<hr>
</center>

### Importing Libraries

In [ ]:
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install graphviz

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import graphviz
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# FOR GOOGLE UPLOAD
#from google.colab import files
#import io
#uploaded = files.upload()
#df = pd.read_csv(io.BytesIO(uploaded["data - data.csv"))

### Loading Dataset

In [ ]:
df = pd.read_csv("/home/yash/B-Tech/SEM-6 (laptop)/PM_Predictive Modeling/PM_Project/DASS_data_21_02_19/data - data.csv")
incorrectEntries = df[(df['VCL6'] == 1) | (df['VCL9'] == 1) | (df['VCL12'] == 1)].index
df.drop(incorrectEntries , inplace=True)
print(df.head())
print("Shape of Dataframe: ",df.shape)

   Q1A  Q1I   Q1E  Q2A  Q2I   Q2E  Q3A  Q3I   Q3E  Q4A  ...  screensize   
0    4   28  3890    4   25  2122    2   16  1944    4  ...           1  \
1    4    2  8118    1   36  2890    2   35  4777    3  ...           2   
2    3    7  5784    1   33  4373    4   41  3242    1  ...           2   
3    2   23  5081    3   11  6837    2   37  5521    1  ...           2   
4    2   36  3215    2   13  7731    3    5  4156    4  ...           2   

   uniquenetworklocation  hand  religion  orientation  race  voted  married   
0                      1     1        12            1    10      2        1  \
1                      1     2         7            0    70      2        1   
2                      1     1         4            3    60      1        1   
3                      1     2         4            5    70      2        1   
4                      2     3        10            1    10      2        1   

   familysize       major  
0           2         NaN  
1           4     

### Preprocessing Data

#### Categorizing Age field into Age-Category

In [ ]:
def categorize_age(age):
  if age<=12:
    return 1
  elif age>=13 and age<=19:
    return 2
  elif age>=20 and age<=30:
    return 3
  elif age>=31 and age<=40:
    return 4
  elif age>=41 and age<=65:
    return 5
  elif age>=66:
    return 6
  else:
    return -99
df['age'] = df['age'].apply(categorize_age)

#### Scaling of Race field

In [ ]:
def scaling_race(race):
  return int(race/10)

df['race'] = df['race'].apply(scaling_race)

#### Handling outliers of Family Size field

In [ ]:
def handling_family_size(size):
  if size>13:
    return 13
  else:
    return size

df['familysize'] = df['familysize'].apply(handling_family_size)

#### Deriving total sum of the scores of Depression, Anxiety & Stress

In [ ]:
depression_ques = ['Q3A','Q5A','Q10A','Q13A','Q16A','Q17A','Q21A','Q24A','Q26A','Q31A','Q34A','Q37A','Q38A','Q42A']
anxiety_ques = ['Q2A','Q4A','Q7A','Q9A','Q15A','Q19A','Q20A','Q23A','Q25A','Q28A','Q30A','Q36A','Q40A','Q41A']
stress_ques = ['Q1A','Q6A','Q8A','Q11A','Q12A','Q14A','Q18A','Q22A','Q27A','Q29A','Q32A','Q33A','Q35A','Q39A']

df['D_total_sum'] = df[depression_ques].sum(axis=1)
df['A_total_sum'] = df[anxiety_ques].sum(axis=1)
df['S_total_sum'] = df[stress_ques].sum(axis=1)

#### Labeling the scores of Depression, Anxiety & Stress

In [ ]:
# for labeling Depression
def depression_cate(dep):
  if dep >= 0 and dep <= 9:
    return 1
  elif dep >= 10 and dep <=13:
    return 2
  elif dep >= 14 and dep <=20:
    return 3
  elif dep >= 21 and dep <=27:
    return 4
  elif dep >= 28:
    return 5
  else:
    return -99

df['D_labels'] = df['D_total_sum'].apply(depression_cate)

# for labeling Anxiety
def anxiety_cate(anx):
  if anx >= 0 and anx <= 7:
    return 1
  elif anx >= 8 and anx <=9:
    return 2
  elif anx >= 10 and anx <=14:
    return 3
  elif anx >= 15 and anx <=19:
    return 4
  elif anx >= 20:
    return 5
  else:
    return -99

df['A_labels'] = df['A_total_sum'].apply(anxiety_cate)

# for labeling Stress
def stress_cate(sts):
  if sts >= 0 and sts <= 14:
    return 1
  elif sts >= 15 and sts <=18:
    return 2
  elif sts >= 19 and sts <=25:
    return 3
  elif sts >= 26 and sts <=33:
    return 4
  elif sts >= 34:
    return 5
  else:
    return -99

df['S_labels'] = df['S_total_sum'].apply(stress_cate)

#### Filtering the unnecesaary data

In [ ]:
# build list of removed features to deleted from dataset
removedFeatures = [f'Q{i}E' for i in range(1, 43)] # add feature 'Q1E' to 'Q42E' to be removed
removedFeatures.extend([f'Q{i}I' for i in range(1, 43)]) # add feature 'Q1E' to 'Q42E' to be removed
removedFeatures.extend([f'Q{i}A' for i in range(1, 43)]) # add feature 'Q1A' to 'Q42A' to be removed
removedFeatures.extend([f'VCL{i}' for i in range(1, 17)]) # add feature 'VCL1' to 'VCL16' to be removed
removedFeatures.extend([ 'source', 'introelapse', 'testelapse', 'surveyelapse', 'engnat', 'hand', 'orientation',
    'voted', 'country', 'screensize', 'uniquenetworklocation','major','gender','D_total_sum','A_total_sum','S_total_sum'])

In [ ]:
df = df.drop(removedFeatures,axis=1)

In [ ]:
df.columns

Index(['TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8',
       'TIPI9', 'TIPI10', 'education', 'urban', 'age', 'religion', 'race',
       'married', 'familysize', 'D_labels', 'A_labels', 'S_labels'],
      dtype='object')

In [ ]:
df.shape

(34583, 20)

### Modeling

#### Partitioning the Data

In [ ]:
# for Depression
DX = df.drop(['D_labels'],axis=1)
AX = df.drop(['A_labels'],axis=1)
SX = df.drop(['S_labels'],axis=1)

Dy = df[['D_labels']]
DX_train, DX_test, Dy_train, Dy_test = train_test_split(DX, Dy, test_size=0.3, random_state=1, stratify=Dy)

# for Anxiety
Ay = df[['A_labels']]
AX_train, AX_test, Ay_train, Ay_test = train_test_split(AX, Ay, test_size=0.3, random_state=1, stratify=Ay)

# for Stress
Sy = df[['S_labels']]
SX_train, SX_test, Sy_train, Sy_test = train_test_split(SX, Sy, test_size=0.3, random_state=1, stratify=Sy)

Next step is to perform feature scaling. The reason for doing feature scaling is to make sure that data for different features are in the same range. The StandardScaler class of sklearn.preprocessing is used.

In [ ]:
# for Depression
D_sc = StandardScaler()
D_sc.fit(DX_train)
DX_train_std = D_sc.transform(DX_train)
DX_test_std = D_sc.transform(DX_test)

# for Anxiety
A_sc = StandardScaler()
A_sc.fit(AX_train)
AX_train_std = A_sc.transform(AX_train)
AX_test_std = A_sc.transform(AX_test)

# for Stress
S_sc = StandardScaler()
S_sc.fit(SX_train)
SX_train_std = S_sc.transform(SX_train)
SX_test_std = S_sc.transform(SX_test)

Next step is to instantiate a C&RT (Decision Tree Classifier) and fit the model. The DecisionTreeClassifier class of sklearn.tree module is used.

In [ ]:
# for Depression
D_ct = tree.DecisionTreeClassifier()
D_ct = D_ct.fit(DX_train_std, Dy_train)

# for Anxiety
A_ct = tree.DecisionTreeClassifier()
A_ct = A_ct.fit(AX_train_std, Ay_train)

# for Stress
S_ct = tree.DecisionTreeClassifier()
S_ct = S_ct.fit(SX_train_std, Sy_train)

Finally, it is time to measure the model performance. Here is the code for doing the same:

In [ ]:
# For Depression
Dy_predict = D_ct.predict(DX_test_std)
print("Accuracy score for Depression C & RT Model: {}%".format(metrics.accuracy_score(Dy_test, Dy_predict)*100))

# For Anxiety
Ay_predict = A_ct.predict(AX_test_std)
print("Accuracy score for Anxiety C & RT Model: {}%".format(metrics.accuracy_score(Ay_test, Ay_predict)*100))

# For Stress
Sy_predict = S_ct.predict(SX_test_std)
print("Accuracy score for Stress C & RT Model: {}%".format(metrics.accuracy_score(Sy_test, Sy_predict)*100))

Accuracy score for Depression C & RT Model: 67.46024096385543%
Accuracy score for Anxiety C & RT Model: 81.10843373493975%
Accuracy score for Stress C & RT Model: 56.81927710843373%


#### Exporting the decision tree classifier into PDF

In [ ]:
# For exporting the decision tree in PDF
'''
D_dot_data = tree.export_graphviz(D_ct, out_file=None)
A_dot_data = tree.export_graphviz(A_ct, out_file=None)
S_dot_data = tree.export_graphviz(S_ct, out_file=None)

D_graph = graphviz.Source(D_dot_data)
A_graph = graphviz.Source(A_dot_data)
S_graph = graphviz.Source(S_dot_data)

D_graph.render("D_decision_tree")
A_graph.render("A_decision_tree")
S_graph.render("S_decision_tree")
'''